In [3]:
conda list

# packages in environment at C:\Users\Kunal\anaconda3\envs\Conqui:
Note: you may need to restart the kernel to use updated packages.

#
# Name                    Version                   Build  Channel
absl-py                   2.2.0                    pypi_0    pypi
aiohappyeyeballs          2.6.1                    pypi_0    pypi
aiohttp                   3.11.14                  pypi_0    pypi
aiosignal                 1.3.2                    pypi_0    pypi
annotated-types           0.7.0                    pypi_0    pypi
anyascii                  0.3.2                    pypi_0    pypi
anyio                     4.6.2           py310haa95532_0  
argon2-cffi               21.3.0             pyhd3eb1b0_0  
argon2-cffi-bindings      21.2.0          py310h827c3e9_1  
asttokens                 3.0.0           py310haa95532_0  
async-lru                 2.0.4           py310haa95532_0  
async-timeout             5.0.1                    pypi_0    pypi
attrs                     25.3.0   

In [2]:
pip install timm


  Using cached timm-1.0.15-py3-none-any.whl.metadata (52 kB)
Using cached timm-1.0.15-py3-none-any.whl (2.4 MB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import timm
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# Datasets
train_dataset = datasets.ImageFolder("C:/Users/Kunal/OneDrive/Desktop/Pythom/Practical Example Regerssion/Alzhimer problem/Dataset/Combined Dataset/train", transform=transform)
val_dataset = datasets.ImageFolder("C:/Users/Kunal/OneDrive/Desktop/Pythom/Practical Example Regerssion/Alzhimer problem/Dataset/Combined Dataset/test", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# Model
model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=4)
model = model.to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Training loop
for epoch in range(5):  # Increase epochs as needed
    model.train()
    running_loss = 0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader):.4f}")

# Validation
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for imgs, labels in val_loader:
        imgs = imgs.to(device)
        outputs = model(imgs)
        preds = torch.argmax(outputs, dim=1).cpu()
        all_preds.extend(preds)
        all_labels.extend(labels)

print(classification_report(all_labels, all_preds, target_names=train_dataset.classes))
